In [ ]:
import pandas as pd
from pathlib import Path
import win32com.client as win32

vendas_df = pd.read_excel('Vendas.xlsx')
email_df = pd.read_excel('Emails.xlsx')
lojas_df = pd.read_csv('Lojas.csv',encoding='latin1', sep=';')
display(vendas_df.head())
display(email_df.head())
display(lojas_df.head())

In [ ]:
vendas_df = vendas_df.merge(lojas_df, on='ID Loja')
display(vendas_df)

In [ ]:
grupo_lojas = {}
for loja in lojas_df['Loja']:
    grupo_lojas[loja] = vendas_df.loc[vendas_df['Loja'] == loja, :]
display(grupo_lojas['Iguatemi Campinas']) 

In [ ]:
dia_indicador = vendas_df['Data'].max()
print(dia_indicador)

In [ ]:
backup_arquivo = Path(r'Backup Arquivos Lojas')
arquivos_lojas = backup_arquivo.iterdir()
lista_nomes_lojas = []

for arquivo in arquivos_lojas:
    lista_nomes_lojas.append(arquivo.name)

for loja in grupo_lojas:
    if loja not in lista_nomes_lojas:    
        nova_pasta = backup_arquivo / loja
        nova_pasta.mkdir()
    
    nome_arquivo = f"{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx"
    local_arquivo = backup_arquivo / loja / nome_arquivo
    
    grupo_lojas[loja].to_excel(local_arquivo)

In [ ]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdeprodutos_dia = 4
meta_qtdeprodutos_ano = 120
meta_ticketmedio_dia = 500
meta_ticketmedio_ano = 500

In [ ]:
for loja in grupo_lojas:
  vendas_loja = grupo_lojas[loja]
  vendas_loja_dia = vendas_loja.loc[vendas_loja['Data']==dia_indicador, :]

  #faturamento
  faturamento_ano = vendas_loja['Valor Final'].sum()
  #print(faturamento_ano)
  faturamento_dia = vendas_loja_dia['Valor Final'].sum()
  #print(faturamento_dia)

  #diversidade de produtos
  qtde_produtos_ano = len(vendas_loja['Produto'].unique())
  #print(qtde_produtos_ano)
  qtde_produtos_dia = len(vendas_loja_dia['Produto'].unique())
  #print(qtde_produtos_dia)


  #ticket medio
  valor_venda = vendas_loja.groupby('Código Venda').sum()
  ticket_medio_ano = valor_venda['Valor Final'].mean()
  #print(ticket_medio_ano)
  #ticket_medio_dia
  valor_venda_dia = vendas_loja_dia.groupby('Código Venda').sum()
  ticket_medio_dia = valor_venda_dia['Valor Final'].mean()
  #print(ticket_medio_dia)
  
  
  #enviar o e-mail
  fromaddr = "teste.testeConfig_2@outlook.com"
  toaddr = email_df.loc[email_df['Loja']==loja, 'E-mail'].values[0]
  nome = email_df.loc[email_df['Loja']==loja, 'Gerente'].values[0]
    
  outlook = win32.Dispatch('outlook.application')
  nome = email_df.loc[email_df['Loja']==loja, 'Gerente'].values[0]
  mail = outlook.CreateItem(0)
  mail.To = 'teste.testeConfig@gmail.com'
  mail.Subject = f'OnePage Dia {dia_indicador.day}/{dia_indicador.month} - Loja {loja}'

  if faturamento_dia >= meta_faturamento_dia:
      cor_fat_dia = 'green'
  else:
      cor_fat_dia = 'red'
  if faturamento_ano >= meta_faturamento_ano:
      cor_fat_ano = 'green'
  else:
      cor_fat_ano = 'red'
  if qtde_produtos_dia >= meta_qtdeprodutos_dia:
      cor_qtde_dia = 'green'
  else:
      cor_qtde_dia = 'red'
  if qtde_produtos_ano >= meta_qtdeprodutos_ano:
      cor_qtde_ano = 'green'
  else:
      cor_qtde_ano = 'red'
  if ticket_medio_dia >= meta_ticketmedio_dia:
      cor_ticket_dia = 'green'
  else:
      cor_ticket_dia = 'red'
  if ticket_medio_ano >= meta_ticketmedio_ano:
      cor_ticket_ano = 'green'
  else:
      cor_ticket_ano = 'red'
  
  mail.HTMLBody = f"""
  <p>Bom dia, {nome}</p>

  <p>O resultado de ontem <strong>({dia_indicador.day}/{dia_indicador.month})</strong> da Loja <strong>{loja}</strong> foi:</p>

  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Dia</th>
      <th>Meta Dia</th>
      <th>Cenário Dia</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_dia:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_dia}</td>
      <td style="text-align: center">{meta_qtdeprodutos_dia}</td>
      <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_dia:.2f}</td>
      <td style="text-align: center">R${meta_ticketmedio_dia:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
    </tr>
  </table>
  <br>
  <table>
    <tr>
      <th>Indicador</th>
      <th>Valor Ano</th>
      <th>Meta Ano</th>
      <th>Cenário Ano</th>
    </tr>
    <tr>
      <td>Faturamento</td>
      <td style="text-align: center">R${faturamento_ano:.2f}</td>
      <td style="text-align: center">R${meta_faturamento_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
    </tr>
    <tr>
      <td>Diversidade de Produtos</td>
      <td style="text-align: center">{qtde_produtos_ano}</td>
      <td style="text-align: center">{meta_qtdeprodutos_ano}</td>
      <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
    </tr>
    <tr>
      <td>Ticket Médio</td>
      <td style="text-align: center">R${ticket_medio_ano:.2f}</td>
      <td style="text-align: center">R${meta_ticketmedio_ano:.2f}</td>
      <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
    </tr>
  </table>
  <p>João Medeiros</p>
  """
  # Anexos (pode colocar quantos quiser):
  attachment  = Path.cwd() / backup_arquivo / loja / f'{dia_indicador.day}_{dia_indicador.month}_{loja}.xlsx'
  mail.Attachments.Add(str(attachment))

  mail.Send()
  print('E-mail da Loja {} enviado'.format(loja))